In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import seaborn as sns

from sklearn.externals import joblib
from scipy.sparse import csr_matrix,lil_matrix
from sklearn.preprocessing import normalize
from sklearn.decomposition import TruncatedSVD
%matplotlib inline

In [13]:
labels = pd.read_csv("./AudioSet/class_labels_indices.csv")

In [14]:
# check if word "speaking" belongs to class
labels["display_name"] = labels["display_name"].apply(lambda s : s.lower())
#labels["is_speaking"] = labels["display_name"].apply(lambda s : "speak" in s or "speech" in s)

In [19]:
labels[labels["is_speaking"] == 1]

,index,mid,display_name,is_singing,is_speaking
0,0,/m/09x0r,speech,0,1
1,1,/m/05zppz,"male speech, man speaking",0,1
2,2,/m/02zsn,"female speech, woman speaking",0,1
3,3,/m/0ytgt,"child speech, kid speaking",0,1
4,4,/m/01h8n0,conversation,0,1
5,5,/m/02qldy,"narration, monologue",0,1
6,6,/m/0261r1,babbling,0,1
7,7,/m/0brhx,speech synthesizer,0,1
8,8,/m/07p6fty,shout,0,1
11,11,/m/07sr1lc,yell,0,1


In [18]:
labels[labels["is_singing"] == 1]

,index,mid,display_name,is_singing,is_speaking
27,27,/m/015lz1,singing,1,0
32,32,/t/dd00003,male singing,1,0
33,33,/t/dd00004,female singing,1,0
34,34,/t/dd00005,child singing,1,0
35,35,/t/dd00006,synthetic singing,1,0
137,137,/m/04rlf,music,1,0
138,138,/m/04szw,musical instrument,1,0
139,139,/m/0fx80y,plucked string instrument,1,0
140,140,/m/0342h,guitar,1,0
141,141,/m/02sgy,electric guitar,1,0


In [20]:
## test tf-record reading

import tensorflow as tf
import numpy as np

def readTfRecordSamples(tfrecords_filename):

    record_iterator = tf.python_io.tf_record_iterator(path=tfrecords_filename)

    for string_record in record_iterator:
        example = tf.train.SequenceExample()
        example.ParseFromString(string_record)
        print(example)  # this prints the abovementioned 4 keys but NOT audio_embeddings

        # the first label can be then parsed like this:
        label = (example.features.feature['labels'].int64_list.value[0])
        print('label 1: ' + str(label))

        # this, however, does not work:
        #audio_embedding = (example.features.feature['audio_embedding'].bytes_list.value[0])

readTfRecordSamples('./AudioSet/audioset_v1_embeddings/bal_train/--.tfrecord')

context {
  feature {
    key: "end_time_seconds"
    value {
      float_list {
        value: 40.0
      }
    }
  }
  feature {
    key: "labels"
    value {
      int64_list {
        value: 399
      }
    }
  }
  feature {
    key: "start_time_seconds"
    value {
      float_list {
        value: 30.0
      }
    }
  }
  feature {
    key: "video_id"
    value {
      bytes_list {
        value: "--cB2ZVjpnA"
      }
    }
  }
}
feature_lists {
  feature_list {
    key: "audio_embedding"
    value {
      feature {
        bytes_list {
          value: "_z\223\016e0\235}\372\365j\2513~M\221\205%\265\237 \373\327\210\240\367\232+_\033\210`q=H\377J[\247j\003\230p\377\371a\334\3770\377\177\377*\030\211\251\226\340\000C\370\246:T\000\2177d\253\377|E\177\006\000\000+\000\377\207EO\323\261\321\272\342Mk\225\033\377\036\377\000\034\207\377\374o_\377\233\\C\343Na\000\377\000\000\333q\377\232m\177\271\000\272\000?\377\377\002\276\000"
        }
      }
      feature {
        bytes_list 

AttributeError: 'SequenceExample' object has no attribute 'features'